# Model 4.1: Parallel Encoder Transformer

In [1]:
import torch
import numpy as np

In [2]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [3]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '/mnt/src')

In [4]:
from models.parallel_encoder_model import ParallelEncoderModel, train

## Load ParallelTrajectoryDatasets

In [5]:
from pathlib import Path
from utils.file_io import read_parallel_trajectory_datasets

In [6]:
data_path = Path("../../data/boomer_data_simulation/")

feature_columns = [
    'left_boom_base_yaw_joint', 
    'left_boom_base_pitch_joint',
    'left_boom_main_prismatic_joint',
    'left_boom_second_roll_joint',
    'left_boom_second_yaw_joint',
    'left_boom_top_pitch_joint',
    'left_boom_ee_joint',
    'cable1_property(length,youngsmodule(bend,twist))',
    'cable2_property(length,youngsmodule(bend,twist))',
    'cable3_property(length,youngsmodule(bend,twist))'
]

label_features = [
    ('cable1_lowest_point', np.array([2], dtype=np.int64)),
    ('cable2_lowest_point', np.array([2], dtype=np.int64)),
    ('cable3_lowest_point', np.array([2], dtype=np.int64))
]

normalized_features = [
    ('cable1_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable2_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable3_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64))
]

In [7]:
data_folder = Path("/mnt/data").absolute()
train_set, _, validation_set, _ = read_parallel_trajectory_datasets(data_folder, 0.8, 0, 0.2, window_size=256, 
                                                                  feature_columns=feature_columns, label_features=label_features, 
                                                                  normalized_features=normalized_features)

Reading .csv files: 1it [00:00,  3.19it/s]
/mnt/src/notebooks/../utils/preprocessing.py:132: RuntimeWarning: divide by zero encountered in divide
  return np.where(x_max != x_min, (features - x_min) / (x_max - x_min), 1).astype(dtype=np.float32)
/mnt/src/notebooks/../utils/preprocessing.py:132: RuntimeWarning: invalid value encountered in divide
  return np.where(x_max != x_min, (features - x_min) / (x_max - x_min), 1).astype(dtype=np.float32)


Reshaping dataframe for learning


In [8]:
features, labels = train_set[0] 
print(features.shape, labels.shape)
input_shape, output_shape = features.shape[-1], labels.shape[-1]
num_parallel_trajectories = features.shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_set) + len(validation_set)} data rows!")

torch.Size([1, 256, 16]) torch.Size([1, 256, 3])
Data shape 16 / 3 of total 40 data rows!


## Load parameter, functions and dataloader

In [9]:
tune_path = Path("/mnt/models/transformer/tune").absolute()
tune_path.mkdir(parents=True, exist_ok=True)

## Train the model with optuna hyperparameter tuning

In [10]:
from ray import tune, train as ray_train
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from typing import Dict
from utils.cluster import attach_ray, disconnect_ray
from utils.optimizer import get_optimizer_function, get_learning_rate_scheduler
from utils.activation import get_activation
from utils.loss_functions import get_loss_function
from torch.utils.data import Dataset, DataLoader
import random; random.seed(0)

In [11]:
def parameter_train(parameter: Dict, train_epochs: int, train_set: Dataset, validation_set: Dataset, model_input_shape: int,
                    model_output_shape: int, num_parallel_trajectories: int) -> None:

    # Determ device on the actual worker used for the trail
    device = ("cuda" if torch.cuda.is_available() else "cpu")

    if device != "cuda":
        print("No cuda device found!")

    train_dataloader = DataLoader(train_set, batch_size=parameter["batch_size"], shuffle = True)
    validation_dataloader = DataLoader(validation_set, batch_size=parameter["batch_size"], shuffle = True)

    activation = get_activation(parameter["activation"])

    model = ParallelEncoderModel(
        num_decoders = num_parallel_trajectories,
        num_heads = parameter["model_dim"][1],
        model_dim = parameter["model_dim"][0],
        feedforward_hidden_dim = parameter["feedforward_dim"],
        num_encoder_layers = parameter["num_encoder_layer"],
        output_dim = model_output_shape,
        transformer_dropout = parameter["transformer_dropout"],
        pos_encoder_dropout = parameter["pos_encoder_dropout"],
        activation = activation
    )

    # The model needs to be on the device used for training before instance the optimizer
    model.to(device)

    optimizer = get_optimizer_function(parameter["optimizer"], model, 1)
    lr_scheduler = get_learning_rate_scheduler(optimizer, parameter["model_dim"][0], parameter["warmup_steps"])
    loss_function = get_loss_function()

    _ = train(train_epochs, train_dataloader, validation_dataloader, model, loss_function, optimizer, lr_scheduler, None, device, report_interval=50, tune=True)

In [12]:
num_samples = 2000
num_epochs = 750
grace_period = 5

In [13]:
model_dim_params = []

n_neighbors = [3, 5, 7, 10, 15]
model_dim = [output_shape]

for d in model_dim:
    num_heads = []
    for i in range(2, d + 1):
        if d % i == 0: num_heads.append(i)

    for h in num_heads:
        for n in n_neighbors:
            model_dim_params.append((d, h, n))

In [14]:
start, end = np.log2(256), np.log2(2048)
num_values = 10
feedforward_dim = [int(2 ** (start + i / (num_values - 1) * (end - start))) for i in range(num_values)]

In [15]:
parameter_space = {
    "pos_encoder_dropout": tune.loguniform(0.05, 0.6, base = 2),
    "transformer_dropout": tune.loguniform(0.05, 0.6, base = 2),
    "num_encoder_layer": tune.choice(list(range(1, 10 + 1, 2))),
    "feedforward_dim": tune.choice(feedforward_dim),
    "batch_size": tune.choice(list(range(64, 256, 16))),
    "model_dim_proj_neighbors": tune.choice(model_dim_params),
    "downprojection": tune.choice([True]),
    "optimizer": tune.choice(["adam", "adamw"]),
    "activation": tune.choice(["relu", "gelu"]),
    "warmup_steps": tune.choice(list(range(1000, 4000, 200))),
}

In [16]:
scheduler = ASHAScheduler(
    metric = "loss",
    mode = "min",
    max_t = num_epochs,
    grace_period = grace_period
)

In [17]:
search_alg = OptunaSearch(
    metric = "loss",
    mode = "min"
) 

In [18]:
attach_ray(manager = True)

2023-12-23 10:59:57,885	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.17.0.2:2099...


Caution: Assign node as manager different from project .env file.


ConnectionError: 

In [ ]:
ray_resources_manager = tune.with_resources(
    trainable=lambda param: parameter_train(param, num_epochs, train_set, validation_set, input_shape, output_shape),
    # See: https://stackoverflow.com/questions/58967793/what-is-the-way-to-make-tune-run-parallel-trials-across-multiple-gpus
    resources={ "cpu": 6, "gpu": 0.5 if torch.cuda.is_available() else 0 }
)

tuner = tune.Tuner(
    ray_resources_manager,
    param_space=parameter_space,
    tune_config=tune.TuneConfig(
        scheduler=scheduler,
        search_alg=search_alg,
         num_samples=num_samples
    ),
    run_config = ray_train.RunConfig(
        name = "parallel_transformer_encoder"
    )
)

In [ ]:
results = tuner.fit()

In [ ]:
disconnect_ray()

In [ ]:
# Save as csv file
result_grid = results.get_dataframe()
result_grid.to_csv(tune_path / "trail_grid_.csv")

In [ ]:
best_result = result_grid.iloc[result_grid['loss'].idxmin()].to_dict()
trail_id = best_result['trial_id']

print(f"Trail ID from the best run: {trail_id}")

In [ ]:
print(f"Best trail by loss value {best_result['loss']}", "\n------")
for key in best_result:
    if 'config' in key:
        print(f"Best trail: {key} value {best_result[key]}")